In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")
open_ai_model = "gpt-5-nano-2025-08-07"

Enter Your OpenAI API Key:··········


# Prompt Pipelining Overview


- 🧱 **Modularity**: Mix and match prompt pieces like LEGO blocks.
- 👓 **Readability**: Break down complex prompts into easy bits.
- 🧠**Flexibility**: Craft prompts on-the-go with logic-based assembly.
- 🔄 **Efficiency**: Loop to append for scenarios like few-shot learning.
- 📝✨**Hybrid Construction**: Combine fixed text with variable-filled templates for structure and spontaneity.
- 💬 **Chat-Friendly**: Create conversational prompts by stacking messages.
- 🛠️ **Customizability**: Let users build prompts with their own components.

**String Prompt Pipelining:**

- 🔗 **Sequential Flow**: Link templates or strings in order, starting with a prompt.

Prompt pipelining turns the art of prompt crafting into a modular, efficient process, perfect for those looking to streamline and enhance their prompt design. 💡


In [3]:
from langchain_core.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template("I'm heading to {destination}. ")
    + "Recommend a great {activity} spot!")

prompt

PromptTemplate(input_variables=['activity', 'destination'], input_types={}, partial_variables={}, template="I'm heading to {destination}. Recommend a great {activity} spot!")

In [4]:
prompt = prompt + "\n\nAlso, any local delicacies I should try?"
prompt

PromptTemplate(input_variables=['activity', 'destination'], input_types={}, partial_variables={}, template="I'm heading to {destination}. Recommend a great {activity} spot!\n\nAlso, any local delicacies I should try?")

In [5]:
print(prompt.template)

I'm heading to {destination}. Recommend a great {activity} spot!

Also, any local delicacies I should try?


In [7]:
prompt.format(destination="Dublin", activity="dining")

"I'm heading to Dublin. Recommend a great dining spot!\n\nAlso, any local delicacies I should try?"

### The key differences between prompt pipelining and multi-input prompts are:

🚂 **Composition**: Pipelining links individual prompts into a cohesive journey.

🖼️ **Independence**: Each pipeline component is crafted separately before integration.

🚶‍♂️ **Sequence**: Pipelining lines up components, unlike multi-input prompts that handle inputs collectively.

📋 **Reuse**: Pipelining excels in reusing pieces; multi-input prompts manage multiple data points in one go.

📖 **Outcome**: Pipelining produces a single narrative; multi-input prompts generate a combined result.

🧱 **Construction**: Pipelining is about assembling prompts step by step, while multi-input prompts are about managing various inputs at once.

In short, pipelining is like creating a melody note by note, whereas multi-input prompts are like playing chords, hitting multiple notes simultaneously.


# Use in a chain

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [10]:
llm = ChatOpenAI(model=open_ai_model, temperature=0.75)

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

for chunk in chain.stream({"destination":"Dublin", "activity":"dining"}):
  print(chunk, end="", flush=True)

Dublin has a fantastic dining scene. Here are a few great options to match different vibes and budgets, plus some local bites you should try.

Recommended spot
- Patrick Guilbaud: Dublin’s iconic fine-dining experience. Two Michelin stars, superb Irish-French seasonal tasting menus, impeccable service. Ideal for a special night out. Reserve well in advance.

Other solid options (different vibes)
- Chapter One: Michelin-starred, elegant and modern Irish cooking with beautiful seasonal plates. A refined option if you want something special but not as formal as Patrick Guilbaud.
- The Woollen Mills: Casual, stylish with hearty Irish-leaning dishes and a great sense of place near the Liffey/Ha’penny Bridge. Good for lunch or dinner with a warm atmosphere.
- The Brazen Head: Ireland’s oldest pub (est. 1198) offering classic Irish dishes in a buzzing, historic setting. Great for a lively, traditional dining experience.

A seafood/casual standout
- For a seafood-focused meal in a relaxed sett

In [11]:
prompt = prompt + " How should I greet the locals in a friendly, informal manner?"


In [12]:
prompt.template

"I'm heading to {destination}. Recommend a great {activity} spot!\n\nAlso, any local delicacies I should try? How should I greet the locals in a friendly, informal manner?"

In [13]:
chain = prompt | llm | output_parser

for chunk in chain.stream({"destination":"Dublin", "activity":"dining"}):
  print(chunk, end="", flush=True)

Fantastic city for a bite. Here are some solid picks in different vibes, plus what to try and how to say hello like a local.

Great dining spots (varying budgets)

- Fine dining that’s worth planning around
  - Chapter One (Parnell Street): Michelin-starred Irish cooking with superb ingredients and technique. Book ahead, especially for weekend dinners.

- Modern Irish/elevated casual
  - Fade Street Social (Fade Street): Buzzing spot with a couple of rooms (tapas-style and a main dining room). Great for a lively night out and lots of Irish-sourced dishes.

- Traditional pub with hearty Irish dishes
  - The Brazen Head (abbr: historic pub near the city center): Oldest pub in Dublin, classic Irish dishes, with a warm pub vibe and live music some nights.

- Casual, crowd-pleasing Irish staples
  - The Woollen Mills (Crown Alley, near the Liffey): Modern Irish cooking in a bright, friendly space; good for a relaxed dinner without breaking the bank.

- A good value classic Irish option
  - 

# Example Use Case

In [14]:
class TravelChatbot:
    def __init__(self, base_template):
        self.model = ChatOpenAI(model=open_ai_model, temperature=0.75)
        self.base_prompt = PromptTemplate.from_template(base_template)

    def append_to_prompt(self, additional_text):
        self.base_prompt += additional_text

    def run_chain(self, destination, activity):
        output_parser = StrOutputParser()
        chain = self.base_prompt | self.model | output_parser
        for chunk in chain.stream({"destination":destination, "activity":activity}):
          print(chunk, end="", flush=True)

# Usage
base_template = "I'm heading to {destination}. Recommend a great {activity} spot!"

chatbot = TravelChatbot(base_template)

# Basic prompt
chatbot.run_chain(destination="Dublin", activity="dining")

Dublin has some fantastic dining. If you want a standout experience, here are solid picks across vibes:

- Top fine-dining pick: Patrick Guilbaud (Merrion Street). Ireland’s premier fine-dining restaurant, two Michelin stars, exceptional French-Irish tasting menus, and flawless service. Very popular, so book well in advance.

- If you want Michelin-starred but a bit more intimate and Irish-influenced: Chapter One (Parnell Square). Elegant modern Irish cuisine, excellent tasting menu, great wine program. Also best to reserve ahead.

- For a classic Dublin experience with great Irish dishes in a relaxed setting: The Winding Stair (near Ha’penny Bridge) or Gallagher’s Boxty House (Temple Bar). Great atmosphere, quality Irish cooking, good value for the area.

If you share your budget, area you’d like (city center, near Trinity, docklands, etc.), and what vibe you’re after (romantic, pre-theatre, family-friendly, etc.), I can tailor a precise pick and help with reservations.

In [15]:
# Append more to the prompt and run again
chatbot.append_to_prompt("\n\nAlso, any local delicacies I should try?")

chatbot.run_chain(destination="Dublin", activity="dining")

Fantastic city for eating. Here are solid picks in different vibes, plus some must-try Irish specialties.

Top pick (great for a memorable splurge)
- Patrick Guilbaud: Ireland’s premier fine-dining spot, with elegant French-Irench cuisine and a strong emphasis on seasonal Irish produce. Reservations are essential, and it’s a special-occasion kind of place. Expect a refined atmosphere and a tasting-menu experience or highly crafted à la carte.

Good alternatives if you want something brilliant but with a different vibe
- The Brazen Head: Dublin’s oldest pub, famous for a cozy, authentic Irish pub dining experience. Great value for classic Irish dishes like Irish stew, coddle, and boxty, in a historic setting with a lively atmosphere.
- The Winding Stair: A charming, sturdy Irish restaurant in the city center near the river. Modern Irish cooking with a comfortable, bookish vibe; good for a relaxed yet special dinner.
- Chapter One: Michelin-starred and minus the formality of a two-Michel

In [16]:
chatbot.append_to_prompt(" How should I greet the locals in a friendly, informal, jolly colloquial manner?")

chatbot.run_chain(destination="Dublin", activity="dining")

Fantastic—Dublin has a—Dublin has a fantastic dining scene. fantastic dining scene. Here are Here are some good some good options plus options plus what to what to try and try and a few a few tips for tips for chatting with chatting with locals.

 locals.

Great diningGreat dining spot ( spot (top picktop pick)
-)
- Chapter One Chapter One (Par (Parnellnell Street Street)
 )
  - Why - Why go: go: A landmark A landmark in Dublin in Dublin’s dining’s dining scene, scene, Michelin-star Michelin-starred withred with superb seasonal superb seasonal Irish cooking Irish cooking and impeccable and impeccable service. It’s service. It’s modern Irish modern Irish with beautifully with beautifully balanced flavors balanced flavors.
 .
  - V - Vibe/ibe/price: Elegant,price: Elegant, refined; refined; €€ €€€+.€+. Reservations are Reservations are essential.
 essential.
  -  - Tip: Tip: Book well Book well in advance in advance and plan and plan a tasting menu or a tasting menu or an à an à la carte

# Chat Prompt Pipeline

🧩 **Composition**: Chat prompt pipelining turns reusable message blocks into a complete conversation flow.

🛠️ **Versatility**: Mix and match static messages with dynamic templates for a custom dialogue.

🔗 **End Result**: You get a ChatPromptTemplate that's ready for action, crafted from your message lineup.

🏗️ **Modularity**: Like using building blocks, this method lets you construct prompts piece by piece for maximum flexibility.

In essence, chat prompt pipelining is about assembling conversations from logical blocks, creating a user-friendly and adaptable ChatPromptTemplate.


In [17]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Setting the scene with a West of Ireland-themed system message
prompt = SystemMessage(content="Welcome to the West of Ireland Chat! 🇮🇪")

# Constructing a chat flow with Irish wit
new_prompt = (
    prompt
    + HumanMessage(content="How's the form?")
    + AIMessage(content="Ah sure, not too bad at all. Did you hear about the Galway rain?")
    + "{input}"
)

# Formatting the chat with the user's response
new_prompt.format_messages(input="No, what about it?")

print(new_prompt)

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessage(content='Welcome to the West of Ireland Chat! 🇮🇪', additional_kwargs={}, response_metadata={}), HumanMessage(content="How's the form?", additional_kwargs={}, response_metadata={}), AIMessage(content='Ah sure, not too bad at all. Did you hear about the Galway rain?', additional_kwargs={}, response_metadata={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [18]:
new_prompt.messages

[SystemMessage(content='Welcome to the West of Ireland Chat! 🇮🇪', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="How's the form?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Ah sure, not too bad at all. Did you hear about the Galway rain?', additional_kwargs={}, response_metadata={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]

In [22]:
model = ChatOpenAI(model=open_ai_model, temperature=0.3)

output_parser = StrOutputParser()

chain = new_prompt | model | output_parser

# Running the chatbot to get the response (using invoke instead of stream)
response = chain.invoke({"input":"No, what about it?"})
print(response)

Ah, the Galway rain—that’s the local seasoning. It can come on in a quick shower, soak you in a heartbeat, and then vanish as if nothing happened, leaving you with a sunbreak and a double take. Locals just roll with it: a good waterproof jacket, a comfy pair of shoes, and a bit of patience.

If you’re out and about, a sheltered cafe or a snug pub on Shop Street is never far away. Want a few shelter spots or a couple of Galway-weather tips for the weekend?
